In [ ]:
pip install pyspark

In [1]:
import sys
import pyspark
from pyspark.sql import SparkSession
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.conf import SparkConf

# Client ID and Secret for the root principal in Polaris
client_id = '31f0fb0a62307999'
client_secret = '792dadce3f3e05219c03340d29d7f09f'
POLARIS_URI = 'http://polaris:8181/api/catalog'
POLARIS_CATALOG_NAME = 'polaris_demo'
POLARIS_WAREHOUSE_DIR = 's3://polaris-test-data-lake/polaris_test/'
POLARIS_CREDENTIALS = f'{client_id}:{client_secret}'
POLARIS_SCOPE = 'PRINCIPAL_ROLE:ALL'

# Arguments for the job
args = {
    "JOB_NAME": "Polaris_Test"
}

# Configure SparkConf with custom settings
conf = (
    pyspark.SparkConf()
        .setAppName('Polaris_Test')
  		#packages
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.6.1,org.apache.iceberg:iceberg-aws:1.6.1,org.apache.hadoop:hadoop-aws:3.4.0,software.amazon.awssdk:bundle:2.23.19,software.amazon.awssdk:url-connection-client:2.23.19')
  		#SQL Extensions
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
        #Configuring Catalog
        .set('spark.sql.catalog.polaris', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.polaris.warehouse', POLARIS_CATALOG_NAME)
        .set('spark.sql.warehouse.dir', POLARIS_WAREHOUSE_DIR)
        .set("spark.sql.catalog.polaris.s3.region", "eu-west-1")
        .set('spark.sql.catalog.polaris.catalog-impl', 'org.apache.iceberg.rest.RESTCatalog')
        .set('spark.sql.catalog.solid_dose.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
        .set('spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation', 'true')
        .set('spark.sql.catalog.polaris.uri', POLARIS_URI)
        .set('spark.sql.catalog.polaris.credential', POLARIS_CREDENTIALS)
        .set('spark.sql.catalog.arctic.authentication.type','BEARER')
        .set('spark.sql.catalog.polaris.scope', POLARIS_SCOPE)
        .set('spark.sql.catalog.polaris.token-refresh-enabled', 'true')
        .set("spark.eventLog.enabled", "true")
        # .set('spark.sql.defaultCatalog','polaris')
)

# Initialize Spark Session using SparkConf
spark = SparkSession.builder \
    .appName("GlueJob") \
    .config(conf=conf) \
    .getOrCreate()

# Initialize GlueContext
glueContext = GlueContext(spark.sparkContext)

# spark = glueContext.spark_session

# Initialize Job in Glue
job = Job(glueContext)
job.init(args["JOB_NAME"], args)

print("Spark Running")

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


:: loading settings :: url = jar:file:/home/glue_user/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/glue_user/.ivy2/cache
The jars for the packages stored in: /home/glue_user/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.4_2.12 added as a dependency
org.apache.iceberg#iceberg-aws added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9afd3582-f072-4bfa-9654-9ac5c48a5b3c;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.6.1 in central
	found org.apache.iceberg#iceberg-aws;1.6.1 in central
	found org.apache.iceberg#iceberg-api;1.6.1 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found org.apache.iceberg#iceberg-bundled-guava;1.6.1 in central
	found org.apache.iceberg#iceberg-common;1.6.1 in central
	found org.apache.iceberg#iceberg-core;1.6.1 in central
	found org.apache.avro#avro;1.11.3 in central
	found c

Exception while setting endpoint config: java.lang.NullPointerException
Spark Running


In [ ]:
spark.stop()

In [ ]:
print(f"Spark version: {spark.version}")

In [2]:
spark.sql("SHOW CATALOGS").show()

24/09/07 01:29:01 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+-------+
|catalog|
+-------+
+-------+



# Use Polaris
Tell spark to use polaris

In [4]:
# spark.sql("USE polaris")
spark.sql("SHOW NAMESPACES").show()

24/09/07 01:29:17 INFO HiveConf: Found configuration file file:/home/glue_user/spark/conf/hive-site.xml
24/09/07 01:29:26 WARN EC2MetadataUtils: Unable to retrieve the requested metadata (/latest/dynamic/instance-identity/document). Failed to connect to service endpoint: 
com.amazonaws.SdkClientException: Failed to connect to service endpoint: 
	at com.amazonaws.internal.EC2ResourceFetcher.doReadResource(EC2ResourceFetcher.java:100)
	at com.amazonaws.internal.EC2ResourceFetcher.doReadResource(EC2ResourceFetcher.java:70)
	at com.amazonaws.internal.InstanceMetadataServiceResourceFetcher.readResource(InstanceMetadataServiceResourceFetcher.java:75)
	at com.amazonaws.internal.EC2ResourceFetcher.readResource(EC2ResourceFetcher.java:66)
	at com.amazonaws.util.EC2MetadataUtils.getItems(EC2MetadataUtils.java:407)
	at com.amazonaws.util.EC2MetadataUtils.getData(EC2MetadataUtils.java:376)
	at com.amazonaws.util.EC2MetadataUtils.getData(EC2MetadataUtils.java:372)
	at com.amazonaws.util.EC2Metadata

+--------------+
|     namespace|
+--------------+
|afrocentric_db|
|       default|
+--------------+



# Create Nested Namespaces

In [ ]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS cloudza_test")
spark.sql("USE CLOUDZA_TEST")
spark.sql("CREATE NAMESPACE IF NOT EXISTS cloudza_test.public")
spark.sql("SHOW NAMESPACES IN cloudza_test").show()

# Create a table

In [ ]:
spark.sql("USE NAMESPACE cloudza_test")
spark.sql("""CREATE TABLE IF NOT EXISTS cloudza_test.test_table(
    id bigint NOT NULL COMMENT 'unique id',
    data string)
USING iceberg;
""")

In [ ]:
spark.sql("SHOW TABLES")

# It's Empty

In [ ]:
spark.sql("USE NAMESPACE cloudza_test")
spark.sql("SELECT * FROM cloudza_test.test_table")

# Insert Data

In [ ]:
spark.sql("SHOW TABLES;")

In [ ]:
spark.sql("INSERT INTO TEST_TABLE VALUES (1, 'some data'), (2, 'more data'), (3, 'yet more data')")
spark.sql("SELECT * FROM TEST_TABLE").show()

In [ ]:
job.commit()